In [1]:
from lsstseries import Ensemble
from lsstseries.analysis.structurefunction2 import calc_sf2
from lsstseries.analysis.stetsonj import calc_stetson_J
import numpy as np
import hipscat
import dask.dataframe as dd
import pandas as pd
from dask.distributed import Client
import multiprocessing;multiprocessing.cpu_count()

96

In [3]:
ens = Ensemble()
ens = ens.from_parquet("/data/epyc/users/brantd/timeseries/ztf_testdata/*.parquet",
                      id_col="ps1_objid",
                      time_col="mjd",
                      flux_col="mag",
                      err_col="magerr")

ens.dropna(1).prune(100)


/astro/users/brantd/.conda/envs/py310/lib/python3.10/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 44498 instead
  warnings.warn(


In [4]:
ens.client_info()

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:44498/status,
Dashboard: http://127.0.0.1:44498/status,Workers: 12
Total threads: 96,Total memory: 0.98 TiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:43837,Workers: 12
Dashboard: http://127.0.0.1:44498/status,Total threads: 96
Started: 3 minutes ago,Total memory: 0.98 TiB
Comm: tcp://127.0.0.1:46583,Total threads: 8
Dashboard: http://127.0.0.1:39295/status,Memory: 83.97 GiB
Nanny: tcp://127.0.0.1:39620,


In [5]:
res = ens.batch(calc_stetson_J, band_to_calc='g')

In [5]:
res = ens.batch(calc_sf2, sthresh=50, band_to_calc=['g','r'])

/data/epyc/users/brantd/lsstseries/src/lsstseries/ensemble.py:211: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  batch = self._data.map_partitions(lambda x: x.groupby(id_col, group_keys=False).apply(
/data/epyc/users/brantd/lsstseries/src/lsstseries/ensemble.py:211: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., gr

In [5]:
res.to_csv("out.csv")

In [10]:
len(res['lc_id'].unique())

203192

In [5]:
res

ps1_objid
74420597561988984      {'g': 0.6647264}
74420599952839167     {'g': 0.29402572}
74420606343319964      {'g': 0.3331358}
74420664289979622      {'g': 1.0316911}
74420666002478050    {'g': -0.28180474}
                            ...        
84590674993150565     {'g': 0.10141596}
84590675195062274      {'g': 0.7435742}
84590675217217666    {'g': -0.12472639}
84600675154817484    {'g': -0.09470266}
84610675136421586    {'g': -0.11108887}
Name: ps1_objid, Length: 203192, dtype: object